In [2]:
import csv
import pandas as pd
import numpy as np
# import geopandas as gpd
import networkx as nx
import math
import pickle

from datetime import datetime as dt
from collections import Counter
from scipy.spatial.distance import cdist
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm


In [4]:
a = np.arange(24).reshape(6,4)
b = [0, 2]

In [7]:
c = a.reshape(3,2,4)

In [8]:
c

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [9]:
c[b, :, :]

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [9]:
tmc_xd_join = pd.read_csv("./data/tmc_xd_join.csv").dropna(subset={"XDSegID"})  # contains all new_out_tmc and new_in_tmc
tmc_xd_join.XDSegID = tmc_xd_join.XDSegID.apply(int).apply(str)
tmc_xd_join = tmc_xd_join.rename(columns={"id":"id_tmc", "XDSegID":"id_xd"})

In [16]:
tmc_xd_join

,id_tmc,OID,id_xd,PreviousXD,NextXDSegI,FRC,RoadNumber,RoadName,LinearID,Country,...,SlipRoad,SpecialRoa,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID
0,104P06676,NaN,388051958,NaN,NaN,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68528,-80.09392,40.68540,-80.09267,E,1970621.0,4326.0
1,104P06676,NaN,429090769,1.310324e+09,1.310291e+09,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68540,-80.09267,40.68543,-80.09180,E,514061.0,4326.0
2,104P06676,NaN,1310323857,1.310598e+09,4.290908e+08,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68528,-80.09392,40.68540,-80.09267,E,514061.0,4326.0
3,104P06676,NaN,1310330692,NaN,1.310331e+09,0.0,NaN,NaN,NaN,United States,...,1.0,NaN,NaN,40.68480,-80.09259,40.68540,-80.09267,N,3461167.0,4326.0
4,104P06676,NaN,1310334146,1.310597e+09,1.310323e+09,0.0,NaN,NaN,NaN,United States,...,1.0,NaN,NaN,40.68677,-80.09629,40.68241,-80.09592,S,3463008.0,4326.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,104+09201,NaN,1310398365,1.310321e+09,1.310254e+09,1.0,19.0,US 19,NaN,United States,...,0.0,NaN,US 19,40.56355,-80.03885,40.55419,-80.03847,S,531444.0,4326.0
1440,104+09201,NaN,1310514659,1.310346e+09,4.290913e+08,2.0,NaN,Ingomar Road,NaN,United States,...,0.0,NaN,Ingomar Road,40.58203,-80.03812,40.58347,-80.03268,E,520633.0,4326.0
1441,104+09201,NaN,1310514675,4.290913e+08,1.310346e+09,2.0,NaN,Ingomar Road,NaN,United States,...,0.0,NaN,Ingomar Road,40.58347,-80.03268,40.58203,-80.03812,W,2507086.0,4326.0
1442,104+09201,NaN,1310607155,NaN,1.310414e+09,3.0,NaN,Highland Road,NaN,United States,...,0.0,NaN,"Old Perry Highway,Highland Road",40.56108,-80.03940,40.56617,-80.04335,N,3484122.0,4326.0


In [11]:
tmc_xd_join.to_csv("./data/tmc_xd_join.csv", index=False)

In [102]:
tmc_xd_join = pd.read_csv("./data/tmc_xd_join.csv")

In [104]:
tmc_xd_join.head(5)

,id_tmc,OID,id_xd,PreviousXD,NextXDSegI,FRC,RoadNumber,RoadName,LinearID,Country,...,SlipRoad,SpecialRoa,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID
0,104P06676,NaN,388051958,NaN,NaN,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68528,-80.09392,40.68540,-80.09267,E,1970621.0,4326.0
1,104P06676,NaN,429090769,1.310324e+09,1.310291e+09,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68540,-80.09267,40.68543,-80.09180,E,514061.0,4326.0
2,104P06676,NaN,1310323857,1.310598e+09,4.290908e+08,0.0,228.0,PA 228,NaN,United States,...,0.0,NaN,PA 228,40.68528,-80.09392,40.68540,-80.09267,E,514061.0,4326.0
3,104P06676,NaN,1310330692,NaN,1.310331e+09,0.0,NaN,NaN,NaN,United States,...,1.0,NaN,NaN,40.68480,-80.09259,40.68540,-80.09267,N,3461167.0,4326.0
4,104P06676,NaN,1310334146,1.310597e+09,1.310323e+09,0.0,NaN,NaN,NaN,United States,...,1.0,NaN,NaN,40.68677,-80.09629,40.68241,-80.09592,S,3463008.0,4326.0


In [105]:
tmc_to_xd = tmc_xd_join[["id_tmc", "id_xd"]]

In [106]:
tmc_to_xd

,id_tmc,id_xd
0,104P06676,388051958
1,104P06676,429090769
2,104P06676,1310323857
3,104P06676,1310330692
4,104P06676,1310334146
...,...,...
1410,104+09201,1310398365
1411,104+09201,1310514659
1412,104+09201,1310514675
1413,104+09201,1310607155


In [51]:
xd_attr = pd.read_csv("./data/shapefiles/xd_attr.csv")
xd_attr.XDSegID = xd_attr.XDSegID.apply(int).apply(str)
xd_attr.PreviousXD = xd_attr.PreviousXD.apply(lambda x: str(int(x)) if not np.isnan(x) else x)
xd_attr.NextXDSegI = xd_attr.NextXDSegI.apply(lambda x: str(int(x)) if not np.isnan(x) else x)
xd_attr = xd_attr.rename(columns={"XDSegID":"id_xd", "PreviousXD":"id_xd_prev", "NextXDSegI":"id_xd_next", "Miles":"miles_xd"})

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_57755/2537207207.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  xd_attr = pd.read_csv("./data/shapefiles/xd_attr.csv")


In [52]:
xd_attr.columns

Index(['OID', 'id_xd', 'id_xd_prev', 'id_xd_next', 'FRC', 'RoadNumber',
       'RoadName', 'LinearID', 'Country', 'State', 'County', 'District',
       'miles_xd', 'Lanes', 'SlipRoad', 'SpecialRoa', 'RoadList', 'StartLat',
       'StartLong', 'EndLat', 'EndLong', 'Bearing', 'XDGroup', 'ShapeSRID'],
      dtype='object')

In [53]:
xd_attr

,OID,id_xd,id_xd_prev,id_xd_next,FRC,RoadNumber,RoadName,LinearID,Country,State,...,SlipRoad,SpecialRoa,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID
0,NaN,128218004,1282656647,1282656526,1,62.0,US 62,NaN,United States,Pennsylvania,...,0,NaN,US 62,41.82235,-79.26444,41.81502,-79.27155,S,266075,4326
1,NaN,128214080,129362453,129006856,3,NaN,Hickory Bridge,NaN,United States,Pennsylvania,...,0,NaN,Hickory Bridge,41.84497,-79.14943,41.84303,-79.15024,S,265976,4326
2,NaN,128244210,429512463,429524197,1,62.0,US 62,NaN,United States,Pennsylvania,...,0,NaN,US 62,41.84243,-79.25704,41.83915,-79.25184,E,266075,4326
3,NaN,128243311,NaN,1282847122,4,NaN,Glen Avenue,NaN,United States,Pennsylvania,...,0,NaN,Glen Avenue,39.96943,-75.56340,39.96953,-75.56338,N,266914,4326
4,NaN,128247637,NaN,NaN,4,NaN,West Washington Street,NaN,United States,Pennsylvania,...,0,NaN,West Washington Street,40.61122,-75.48515,40.61137,-75.48460,E,267041,4326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117674,NaN,1310618675,1310390374,389122055,1,NaN,East Orange Street,NaN,United States,Pennsylvania,...,0,NaN,"PA 462,East Orange Street",40.04001,-76.28734,40.03976,-76.30393,W,2008848,4326
117675,NaN,1310618793,1310618797,1310618559,2,403.0,PA 403,NaN,United States,Pennsylvania,...,0,NaN,PA 403,40.28836,-78.91395,40.29468,-78.91671,N,2010740,4326
117676,NaN,1310618797,1310418378,1310618793,2,403.0,PA 403,NaN,United States,Pennsylvania,...,0,NaN,PA 403,40.28616,-78.91973,40.28836,-78.91395,E,2010740,4326
117677,NaN,1310618822,NaN,NaN,4,NaN,NaN,NaN,United States,Pennsylvania,...,0,NaN,NaN,40.79099,-76.53468,40.79094,-76.53272,E,3625732,4326


In [54]:
xd_attr.to_csv("./data/shapefiles/xd_attr.csv", index=False)

In [58]:
xd_attr.loc[:,["id_xd", "id_xd_prev", "id_xd_next", "miles_xd"]]

,id_xd,id_xd_prev,id_xd_next,miles_xd
0,128218004,1282656647,1282656526,0.628703
1,128214080,129362453,129006856,0.141113
2,128244210,429512463,429524197,0.356400
3,128243311,NaN,1282847122,0.006618
4,128247637,NaN,NaN,0.030839
...,...,...,...,...
117674,1310618675,1310390374,389122055,0.971595
117675,1310618793,1310618797,1310618559,0.649072
117676,1310618797,1310418378,1310618793,0.700093
117677,1310618822,NaN,NaN,0.102943


In [60]:
core_xd_attr = xd_attr.loc[:,["id_xd", "id_xd_prev", "id_xd_next", "miles_xd"]]
core_xd_attr = core_xd_attr.set_index(keys="id_xd")

In [61]:
core_xd_attr

,id_xd_prev,id_xd_next,miles_xd
id_xd,,,
128218004,1282656647,1282656526,0.628703
128214080,129362453,129006856,0.141113
128244210,429512463,429524197,0.356400
128243311,NaN,1282847122,0.006618
128247637,NaN,NaN,0.030839
...,...,...,...
1310618675,1310390374,389122055,0.971595
1310618793,1310618797,1310618559,0.649072
1310618797,1310418378,1310618793,0.700093


In [64]:
core_xd_attr.to_csv("./data/shapefiles/core_xd_attr.csv")

In [66]:
dict_xd_attr = core_xd_attr.to_dict(orient="index")  # key: <id_xd>; value: dict() with keys: 'id_xd_prev', 'id_xd_next', 'miles_xd'

In [92]:
str(dict_xd_attr["128243311"]['id_xd_prev'])

'nan'

In [68]:
next_xd = {}

In [94]:
for xd in tqdm(dict_xd_attr):
    dist = 5 # 5 miles
    curr = xd
    nxt = dict_xd_attr[curr]['id_xd_next']
    next_list = []
    while dist >= 0 and str(nxt) != "nan":
        next_list.append(nxt)
        if nxt not in dict_xd_attr:
            break
        else:
            dist -= dict_xd_attr[nxt]["miles_xd"]
            nxt = dict_xd_attr[nxt]['id_xd_next']
    next_xd[xd] = next_list

100%|██████████| 117679/117679 [00:00<00:00, 183256.67it/s]


In [ ]:
next_xd

In [97]:
with open("./data/next_xd.pkl", "wb") as f:
    pickle.dump(next_xd, f)

In [99]:
with open("./data/next_xd.pkl", "rb") as f:
    temp = pickle.load(f)

In [100]:
temp

{'128218004': ['1282656526',
  '1282656568',
  '1282656798',
  '1282656835',
  '387477309',
  '429524193',
  '429524194',
  '429524195'],
 '128214080': ['129006856', '1282696696'],
 '128244210': ['429524197',
  '429540393',
  '429524192',
  '1282656606',
  '1282656647',
  '128218004',
  '1282656526',
  '1282656568',
  '1282656798',
  '1282656835',
  '387477309'],
 '128243311': ['1282847122',
  '1282639653',
  '387520623',
  '1282662727',
  '1282561856',
  '1282561970',
  '387586574',
  '1282645373',
  '1282658954',
  '429519260',
  '1282785833',
  '1282645354'],
 '128247637': [],
 '128249378': ['1282693275'],
 '128249379': ['128249378', '1282693275'],
 '128249487': [],
 '128256044': [],
 '128264215': ['1282807901', '1282758647', '1282628412', '1282572680'],
 '128264355': ['1282645740', '1282767620', '1282740563'],
 '128266250': [],
 '128267870': ['129192777', '1282816566'],
 '128274683': [],
 '128274595': ['1282845829',
  '1282731849',
  '1282579564',
  '1282731940',
  '387516067',
  '

In [64]:
thres_accu_recall = np.load("/Users/haowu/Desktop/Transportation Research/Traffic Prediction/inference/thres_accu_recall.npy")

In [65]:
thres_accu_recall

array([[0.        , 0.05      , 0.1       , 0.15      , 0.2       ,
        0.25      , 0.3       , 0.35      , 0.4       , 0.45      ,
        0.5       , 0.55      , 0.6       , 0.65      , 0.7       ,
        0.75      , 0.8       , 0.85      , 0.9       , 0.95      ],
       [0.02645672, 0.02645672, 0.02645672, 0.02645672, 0.02645672,
        0.02645672, 0.02645672, 0.02645672, 0.02645672, 0.02645672,
        0.02645672, 0.02645781, 0.02692439, 0.06543814, 0.92280233,
        0.97354329, 0.97354329, 0.97354329, 0.97354329, 0.97354329],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.99969095, 0.97034568, 0.08032496,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [41]:
waze_out = np.load("./data/waze_out_tmc.npy")

In [55]:
time_idx = 23*180+129
seg_idx = 18

waze_out[27*180: 28*180, seg_idx]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [56]:
a = np.arange(24).reshape(4,3,2)

In [57]:
a

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15],
        [16, 17]],

       [[18, 19],
        [20, 21],
        [22, 23]]])

In [62]:
np.stack([a]*3, axis=-1)

array([[[[ 0,  0,  0],
         [ 1,  1,  1]],

        [[ 2,  2,  2],
         [ 3,  3,  3]],

        [[ 4,  4,  4],
         [ 5,  5,  5]]],


       [[[ 6,  6,  6],
         [ 7,  7,  7]],

        [[ 8,  8,  8],
         [ 9,  9,  9]],

        [[10, 10, 10],
         [11, 11, 11]]],


       [[[12, 12, 12],
         [13, 13, 13]],

        [[14, 14, 14],
         [15, 15, 15]],

        [[16, 16, 16],
         [17, 17, 17]]],


       [[[18, 18, 18],
         [19, 19, 19]],

        [[20, 20, 20],
         [21, 21, 21]],

        [[22, 22, 22],
         [23, 23, 23]]]])

In [58]:
b = np.ones([4,3,2])

In [60]:
a * (b < 1)

array([[[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]]])

In [40]:
np.mean(b[:2], axis=0)

array([[3., 4.],
       [5., 6.],
       [7., 8.]])

In [17]:
t = torch.arange(24).view(4,3,2)

In [22]:
t/t

tensor([[[nan, 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])

In [21]:
t.sum(axis=0).sum(axis=1)

tensor([ 76,  92, 108])

In [23]:
temp = np.arange(24).reshape(6,2,2)

In [24]:
b = temp * 2

In [25]:
b

array([[[ 0,  2],
        [ 4,  6]],

       [[ 8, 10],
        [12, 14]],

       [[16, 18],
        [20, 22]],

       [[24, 26],
        [28, 30]],

       [[32, 34],
        [36, 38]],

       [[40, 42],
        [44, 46]]])

In [27]:
np.maximum(np.abs(temp), np.abs(b))

array([[[ 0,  2],
        [ 4,  6]],

       [[ 8, 10],
        [12, 14]],

       [[16, 18],
        [20, 22]],

       [[24, 26],
        [28, 30]],

       [[32, 34],
        [36, 38]],

       [[40, 42],
        [44, 46]]])

In [29]:
temp[0,0,0] = 1

In [30]:
(temp-b)/b

/var/folders/1s/gdw_sfh525x0796jh_nnmx6r0000gn/T/ipykernel_43377/826103142.py:1: RuntimeWarning: divide by zero encountered in true_divide
  (temp-b)/b


array([[[ inf, -0.5],
        [-0.5, -0.5]],

       [[-0.5, -0.5],
        [-0.5, -0.5]],

       [[-0.5, -0.5],
        [-0.5, -0.5]],

       [[-0.5, -0.5],
        [-0.5, -0.5]],

       [[-0.5, -0.5],
        [-0.5, -0.5]],

       [[-0.5, -0.5],
        [-0.5, -0.5]]])

In [87]:
def assign_id(dataframe,lon_var,lat_var,shapefile,id_str):
    ddf = dataframe.copy()
    local_gdf = gpd.GeoDataFrame(
            ddf, crs={'init': 'epsg:4326'},
            geometry=[Point(xy) for xy in
                              zip(dataframe[lon_var], dataframe[lat_var])])
    local_gdf = gpd.sjoin(
            local_gdf, shapefile, how='left', op='within')
    return local_gdf[id_str]

def build_tmc_dict(net):
    tmc2edge = { }
    for key in net.edges:
        tmc_id = net.edges[key]['id']
        if tmc_id not in tmc2edge:
            tmc2edge[tmc_id] = [ ]
        tmc2edge[tmc_id].append(key)
    return tmc2edge

def build_waze_incident_matrix(waze, buffer_shp, tmc_header, matrix=None):
    affected_tmcs_df = assign_id(waze, 'x','y',buffer_shp,'id')
    min_date = datetime.date(2019, 2, 10)
    max_date = datetime.date(2019, 7, 24)
    n_days = (max_date-min_date).days+1
    n_ts_per_day = 288
    if matrix is None:
        matrix = np.zeros((n_days*n_ts_per_day, len(tmc_header)))
    n_incidents = len(waze)
    for i in tqdm(range(n_incidents)):
        record = waze.iloc[i]
        start_date = (record.DateTimeEventOccurs.date()-min_date).days
        end_date   = (record.ActualDateTimeOpened.date()-min_date).days
        start_ts = record.DateTimeEventOccurs.hour*12 + record.DateTimeEventOccurs.minute//5
        end_ts  = record.ActualDateTimeOpened.hour*12 + record.ActualDateTimeOpened.minute//5+1
        # select tmcs
        tmc_ids = np.unique(np.array(affected_tmcs_df.loc[i]))
        indices = np.isin(tmc_header, tmc_ids)
        # matrix index
        start_index = max(start_date*n_ts_per_day + start_ts,0)
        end_index =   max(end_date*n_ts_per_day + end_ts,0)
        matrix[start_index:end_index][:,indices] = 1
    return matrix

def fill_tmcs_by_spatial_distance(mat, tmc_header, tmc2edge):
    print('filling waze points with spatial distance...',end='')
    for row in tqdm(mat):
        sel_tmcs = tmc_header[row > 0]
        if len(sel_tmcs) > 1:
            for (bgn_tmc, end_tmc) in permutations(sel_tmcs, 2):
                bgn_node, end_node = tmc2edge[bgn_tmc][0][1], tmc2edge[end_tmc][0][0]
                if nx.has_path(net, end_node, bgn_node):
                    ns = nx.shortest_path(net, end_node, bgn_node)
                    n_nodes = len(ns)
                    if n_nodes == 4:
                        filled_tmc_id = net.edges[ns[1],ns[2]]['id']
                        column_idx = np.where(tmc_header == filled_tmc_id)[0]
                        row[column_idx] = 1
    print('done!')
    return mat

In [88]:
waze = pd.read_pickle('./data/waze_vis.pickle').reset_index(drop=True)
waze = waze[waze.type.isin(['ACCIDENT','JAM'])].reset_index(drop=True)
shp = gpd.read_file('./data/shapefile/cranberry.shp', SHAPE_RESTORE_SHX=True)
net = nx.read_shp('./data/shapefile/cranberry.shp', simplify=True)
tmc2edge = build_tmc_dict(net)

/var/folders/1s/gdw_sfh525x0796jh_nnmx6r0000gn/T/ipykernel_56491/3403020948.py:4: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  net = nx.read_shp('./data/shapefile/cranberry.shp', simplify=True)


In [89]:
road_buffer_shp = shp.to_crs(epsg=3857).buffer(8,resolution=1).to_crs(epsg=4326)
shp['buffer'] = road_buffer_shp
buffer_shp = shp[['id','buffer']]
buffer_shp.columns = ['id','geometry']
# buffer_shp = gpd.GeoDataFrame(buffer_shp,crs={'init': 'epsg:4326'})
buffer_shp = gpd.GeoDataFrame(buffer_shp).to_crs(epsg=4326)
tmc_ids = np.sort(buffer_shp.id.unique())
print(buffer_shp)

            id                                           geometry
0    104P06676  POLYGON ((-80.09267 40.68534, -80.09273 40.685...
1    104P06676  POLYGON ((-80.09448 40.68523, -80.09472 40.685...
2    104N06676  POLYGON ((-80.09602 40.68540, -80.09601 40.685...
3    104P04742  POLYGON ((-80.10031 40.68113, -80.09998 40.679...
4    104P11608  POLYGON ((-80.09594 40.68269, -80.09594 40.683...
..         ...                                                ...
326  104+09197  POLYGON ((-80.03124 40.51800, -80.03101 40.517...
327  104+09198  POLYGON ((-80.03551 40.52553, -80.03544 40.525...
328  104+09199  POLYGON ((-80.03584 40.54413, -80.03584 40.544...
329  104+09200  POLYGON ((-80.03947 40.56039, -80.03951 40.560...
330  104+09201  POLYGON ((-80.03791 40.58169, -80.03791 40.581...

[331 rows x 2 columns]


In [90]:
mat = build_waze_incident_matrix(waze, buffer_shp, tmc_ids)

/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/1s/gdw_sfh525x0796jh_nnmx6r0000gn/T/ipykernel_56491/447766416.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  mat = build_waze_incident_matrix(waze, buffer_shp, tmc_ids)
/var/folders/1s/gdw_sfh525x0796jh_nnmx6r0000gn/T/ipykernel_56491/369826282.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:432

In [91]:
mat.shape

(47520, 315)

In [92]:
mat = fill_tmcs_by_spatial_distance(mat, tmc_ids, tmc2edge)

filling waze points with spatial distance...

100%|██████████| 47520/47520 [00:43<00:00, 1083.49it/s]

done!


In [93]:
mat.shape

(47520, 315)

In [96]:
time_slot = pd.date_range(start='02/10/2019 ', end="07/25/2019", inclusive="left",freq="5min")

In [97]:
time_slot

DatetimeIndex(['2019-02-10 00:00:00', '2019-02-10 00:05:00',
               '2019-02-10 00:10:00', '2019-02-10 00:15:00',
               '2019-02-10 00:20:00', '2019-02-10 00:25:00',
               '2019-02-10 00:30:00', '2019-02-10 00:35:00',
               '2019-02-10 00:40:00', '2019-02-10 00:45:00',
               ...
               '2019-07-24 23:10:00', '2019-07-24 23:15:00',
               '2019-07-24 23:20:00', '2019-07-24 23:25:00',
               '2019-07-24 23:30:00', '2019-07-24 23:35:00',
               '2019-07-24 23:40:00', '2019-07-24 23:45:00',
               '2019-07-24 23:50:00', '2019-07-24 23:55:00'],
              dtype='datetime64[ns]', length=47520, freq='5T')

In [98]:
waze_df = pd.DataFrame(data=mat, columns=tmc_ids, index=time_slot)

In [99]:
waze_df

,104+04438,104+04439,104+04440,104+04441,104+04442,104+04443,104+04444,104+04445,104+04528,104+04529,...,1310574747,1310574829,1310583799,1310593380,1310597255,1310597366,1310597390,1310597496,1310597520,389134964
2019-02-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-24 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-24 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-24 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-24 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
waze_df.to_pickle("./data/waze_df.pickle")

In [74]:
temp = pd.read_pickle("./data/waze_df.pickle")

In [75]:
temp

,104+04438,104+04439,104+04440,104+04441,104+04442,104+04443,104+04444,104+04445,104+04528,104+04529,...,1310574747,1310574829,1310583799,1310593380,1310597255,1310597366,1310597390,1310597496,1310597520,389134964
2019-02-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
temp = np.load("./data/waze_out_xd.npy")

In [105]:
temp, temp.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (147600, 70))

In [ ]:
chunksize = 10 ** 7
file = "data/Cranberry_ritis_1min_class123/manually_select_cranberry_class123_20181101_20190727_dont_average/manually_select_cranberry_class123_20181101_20190727_dont_average.csv"
with pd.read_csv(file, chunksize=chunksize) as reader:
    for chunk in reader:
        

In [78]:
with open("data/Cranberry_ritis_1min_class123/manually_select_cranberry_class123_20181101_20190727_dont_average/manually_select_cranberry_class123_20181101_20190727_dont_average.csv", "r") as input, open("downsampled_XD.csv", "w") as output:
    csv_reader = csv.reader(input)
    header = next(csv_reader, None)
    print(header)
    for row in csv_reader:
        print(row)
        break

['xd_id', 'measurement_tstamp', 'speed', 'average_speed', 'reference_speed', 'travel_time_minutes', 'confidence_score', 'cvalue']
['1310384989', '2018-11-01 00:00:00', '64.00', '64.00', '66.00', '0.60', '30.00', '91.00']


In [115]:
start_date = dt(2019, 2, 10)
end_date = dt(2019, 7, 24)

In [75]:
temp = pd.read_csv("temp.csv")

In [76]:

temp.measurement_tstamp = pd.to_datetime(temp.measurement_tstamp)
print(temp.measurement_tstamp.dt.hour*60 + temp.measurement_tstamp.dt.minute)
temp = temp[(start_date <= temp.measurement_tstamp) & 
            (temp.measurement_tstamp <= end_date) & 
            (temp.measurement_tstamp.dt.hour*60 + temp.measurement_tstamp.dt.minute >= 360) & 
            (temp.measurement_tstamp.dt.hour*60 + temp.measurement_tstamp.dt.minute < 1260)]


0      285
1      345
2      380
3      390
4      395
5      360
6     1260
7      485
8      530
9      545
10     555
11     570
Name: measurement_tstamp, dtype: int64


In [77]:
temp

,tmc_code,measurement_tstamp,speed,average_speed,reference_speed,travel_time_minutes,data_density
5,104N09100,2019-02-11 06:00:00,11.0,10.0,35.0,0.11,A


In [47]:
list = [temp, temp]
final = pd.concat(list)

In [48]:
final

,tmc_code,measurement_tstamp,speed,average_speed,reference_speed,travel_time_minutes,data_density


In [36]:

with open("temp.csv", "r") as input, open("temp_out.csv", "w") as output:
    csv_reader = csv.reader(input)
    csv_writer = csv.writer(output)
    header = next(csv_reader, None)
    csv_writer.writerow(header)
    for row in csv_reader:
        d = dt.strptime(row[1], "%Y-%m-%d %H:%M:%S")
        if start_date <= d <= end_date and 360 <= d.hour*60+d.minute <= 1260:
            csv_writer.writerow(row)


In [26]:
waze_vis = pd.read_pickle("/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Bin/waze_vis.pickle")

In [27]:
waze_vis.head(5)

,DateTimeEventOccurs,ActualDateTimeOpened,street,subtype,type,x,y
27,2019-02-09 17:43:31.055000-05:00,2019-02-09 18:13:00-05:00,SR-228,NaN,ACCIDENT,-80.055461,40.688781
94,2019-02-09 17:45:54.114000-05:00,2019-02-09 18:18:00-05:00,McKnight Rd,JAM_MODERATE_TRAFFIC,JAM,-80.021042,40.554006
126,2019-02-09 17:49:29.683000-05:00,2019-02-09 18:18:00-05:00,Pittsburgh Rd,HAZARD_ON_SHOULDER_CAR_STOPPED,WEATHERHAZARD,-79.934700,40.705969
293,2019-02-09 17:54:43.322000-05:00,2019-02-09 18:28:00-05:00,40th Street Br,JAM_HEAVY_TRAFFIC,JAM,-79.968579,40.472774
354,2019-02-09 16:46:08.560000-05:00,2019-02-09 18:33:00-05:00,I-279 S,HAZARD_ON_SHOULDER_CAR_STOPPED,WEATHERHAZARD,-80.053547,40.517439


In [28]:
waze_vis.tail(5)

,DateTimeEventOccurs,ActualDateTimeOpened,street,subtype,type,x,y
5620691,2019-07-24 22:39:05-04:00,2019-07-24 23:03:00-04:00,SR-28 S,HAZARD_ON_ROAD_CONSTRUCTION,WEATHERHAZARD,-79.912735,40.496224
5620696,2019-07-24 22:31:23-04:00,2019-07-24 23:03:00-04:00,I-79 S,HAZARD_ON_SHOULDER_CAR_STOPPED,WEATHERHAZARD,-80.137745,40.505241
5620765,2019-07-24 22:28:42-04:00,2019-07-24 23:08:00-04:00,I-79 N,HAZARD_ON_SHOULDER_CAR_STOPPED,WEATHERHAZARD,-80.108535,40.582106
5620889,2019-07-24 22:53:14-04:00,2019-07-24 23:23:00-04:00,Mount Nebo Rd,JAM_STAND_STILL_TRAFFIC,JAM,-80.083613,40.547297
5620951,2019-07-24 22:30:08-04:00,2019-07-24 23:28:00-04:00,SR-228,HAZARD_ON_ROAD_OBJECT,WEATHERHAZARD,-80.096161,40.685233


In [29]:
incident_feat = pd.read_csv("/Users/haowu/Desktop/Transportation Research/FHWA/zsl_cranberry/data/incident_features.csv")

In [33]:
print(incident_feat.shape)

(47808, 316)


In [32]:
print(list(incident_feat.columns))

['ts', '104+04438', '104+04439', '104+04440', '104+04441', '104+04442', '104+04443', '104+04444', '104+04445', '104+04528', '104+04529', '104+04530', '104+04531', '104+04532', '104+04533', '104+04540', '104+04541', '104+04542', '104+04543', '104+04636', '104+04637', '104+04639', '104+04640', '104+04641', '104+04642', '104+04643', '104+04644', '104+04734', '104+04735', '104+04736', '104+04737', '104+04738', '104+04739', '104+04740', '104+04741', '104+04742', '104+04782', '104+04783', '104+04784', '104+06224', '104+06225', '104+06226', '104+06227', '104+06228', '104+06229', '104+06230', '104+06676', '104+06678', '104+06679', '104+06680', '104+06681', '104+06682', '104+06683', '104+06684', '104+08678', '104+08679', '104+09013', '104+09014', '104+09015', '104+09017', '104+09018', '104+09194', '104+09195', '104+09196', '104+09197', '104+09198', '104+09199', '104+09200', '104+09201', '104+11466', '104+13682', '104+13683', '104+13684', '104+51754', '104-04438', '104-04439', '104-04440', '104-

In [44]:
np.unique(incident_feat.iloc[:, 1:].values)

array([0., 1., 2.])

In [118]:
out_inc_label = pd.read_csv("/Users/haowu/Desktop/Transportation Research/FHWA/zsl_cranberry/data/incident_labels.csv")

In [109]:
out_inc_label

,measurement_tstamp,104+04438,104+04439,104+04440,104+04441,104+04442,104+04443,104+04444,104+04445,104+04528,...,1310574747,1310574829,1310583799,1310593380,1310597255,1310597366,1310597390,1310597496,1310597520,389134964
0,2019-02-10 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-02-10 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-02-10 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-02-10 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-02-10 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47227,2019-07-23 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47228,2019-07-23 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47229,2019-07-23 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47230,2019-07-23 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
out_inc_label = out_inc_label.set_index("measurement_tstamp")
out_inc_label.index = pd.to_datetime(out_inc_label.index)
out_inc_label = out_inc_label.resample("5 min").asfreq()
out_inc_label = out_inc_label[
                (start_date <= out_inc_label.index) & 
                (out_inc_label.index < end_date) & 
                (out_inc_label.index.hour*60 + out_inc_label.index.minute >= 360) & 
                (out_inc_label.index.hour*60 + out_inc_label.index.minute < 1260) 
                ]

In [126]:
new_out_tmc_xd = set(['1310310475', '1310583799', '1310593380', '104P04441', '104P11466', '104P04739', '104N04441', '104P04735', '104N04541', '104N04742', '1310267020', '104+04543', '104-04442', '104P04540', '104-04742', '104N04540', '104P04740', '104+04542', '104N04444', '1310427817', '1310330692', '104-06680', '104P04736', '1310597255', '104+06678', '104-06679', '104-04737', '104P04742', '104-11466', '104-04736', '1310597390', '1310234750', '104+11466', '1310532417', '1310597496', '1310597520', '104+13683', '104-04738', '104P04445', '1310307852', '104+06679', '104+04444', '104-04741', '104+04445', '104-04540', '1310457035', '104-04443', '104N04542', '104-06678', '104P04542', '1310597366', '104+04742', '104+04737', '104+04740', '1310547738', '104N04736', '104N04443', '104N04442', '104-04735', '1310243607', '1310307897', '1310480559', '104+06681', '104-04441', '104N04735', '104-04444', '104+04736', '104-04542', '104-04739', '104P04541', '1310385586', '104+04540', '104+04738', '104+06680', '104+04739', '104-04541', '1310406313', '104N11466'])

In [139]:
out_inc_label = out_inc_label.loc[:, new_out_tmc_xd]

In [141]:
match_tmc_xd = pd.read_csv("temp_match_target_tmc_xd.csv")

In [150]:
new_out_xd = ['1310310475', '1310583799', '1310593380', '1310597366', '1310267020', '1310547738', '1310243607', '1310307897', '1310480559', '1310427817', '1310330692', '1310597255', '1310385586', '1310597390', '1310234750', '1310532417', '1310597496', '1310597520', '1310406313', '1310307852', '1310457035']

In [153]:
new_col_name = [c if c in new_out_xd else match_tmc_xd[match_tmc_xd.tmc == c]["xd"].values[0] for c in out_inc_label.columns]

In [140]:
out_inc_label

,104-04738,104N04736,104N04542,104P04445,1310307897,104+04543,104N04540,1310547738,104+11466,104-04742,...,104P04541,104-11466,1310597366,104-06678,104N04735,1310593380,1310597255,104N04444,1310243607,104+06681
measurement_tstamp,,,,,,,,,,,,,,,,,,,,,
2019-02-10 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 20:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
out_inc_label.columns = new_col_name

In [156]:
out_inc_label

,441067560,449075259,1310275187,1310495199,1310307897,1310499690,1310384889,1310547738,1310392764,429103830,...,1310499986,1310593317,1310597366,1310487063,449075260,1310593380,1310597255,1310593291,1310243607,449039590
measurement_tstamp,,,,,,,,,,,,,,,,,,,,,
2019-02-10 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-10 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 20:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-23 20:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
temp = pd.read_csv("/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Carnberry_NPMRDS_1min/manually_select_cranberry_2018_dont_average/manually_select_cranberry_2018_dont_average.csv")
temp_2 = pd.read_csv("/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Carnberry_NPMRDS_1min/manually_select_cranberry_2018_dont_average_2/manually_select_cranberry_2018_dont_average.csv")
temp_3 = pd.read_csv("/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Carnberry_NPMRDS_1min/manually_select_cranberry_2018_dont_average_3/manually_select_cranberry_2018_dont_average.csv")

In [21]:
print(temp.head(5))
time = list(temp["measurement_tstamp"].unique())
hms = set([t.split()[1] for t in time])
print(sorted(list(hms)))
print(len(hms))

    tmc_code   measurement_tstamp  speed  average_speed  reference_speed  \
0  104N09100  2018-11-01 04:45:00   14.0           27.0             35.0   
1  104N09100  2018-11-01 05:45:00   14.0           17.0             35.0   
2  104N09100  2018-11-01 06:20:00    8.0           14.0             35.0   
3  104N09100  2018-11-01 06:30:00   18.0           14.0             35.0   
4  104N09100  2018-11-01 06:35:00   14.0           14.0             35.0   

   travel_time_minutes data_density  
0                 0.09            A  
1                 0.09            A  
2                 0.15            A  
3                 0.07            A  
4                 0.09            A  
['00:00:00', '00:05:00', '00:10:00', '00:15:00', '00:20:00', '00:25:00', '00:30:00', '00:35:00', '00:40:00', '00:45:00', '00:50:00', '00:55:00', '01:00:00', '01:05:00', '01:10:00', '01:15:00', '01:20:00', '01:25:00', '01:30:00', '01:35:00', '01:40:00', '01:45:00', '01:50:00', '01:55:00', '02:00:00', '02:05:00', '0

In [22]:
tmc_2019_all = pd.read_csv("/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Carnberry_NPMRDS_1min/manually_select_cranberry_2019_dont_average/manually_select_cranberry_2019_dont_average.csv")

In [24]:
print(tmc_2019_all.shape)
time = list(tmc_2019_all["measurement_tstamp"].unique())
hms = set([t.split()[1] for t in time])
print(sorted(list(hms)))
print(len(hms))

(9136192, 7)
['00:00:00', '00:05:00', '00:10:00', '00:15:00', '00:20:00', '00:25:00', '00:30:00', '00:35:00', '00:40:00', '00:45:00', '00:50:00', '00:55:00', '01:00:00', '01:05:00', '01:10:00', '01:15:00', '01:20:00', '01:25:00', '01:30:00', '01:35:00', '01:40:00', '01:45:00', '01:50:00', '01:55:00', '02:00:00', '02:05:00', '02:10:00', '02:15:00', '02:20:00', '02:25:00', '02:30:00', '02:35:00', '02:40:00', '02:45:00', '02:50:00', '02:55:00', '03:00:00', '03:05:00', '03:10:00', '03:15:00', '03:20:00', '03:25:00', '03:30:00', '03:35:00', '03:40:00', '03:45:00', '03:50:00', '03:55:00', '04:00:00', '04:05:00', '04:10:00', '04:15:00', '04:20:00', '04:25:00', '04:30:00', '04:35:00', '04:40:00', '04:45:00', '04:50:00', '04:55:00', '05:00:00', '05:05:00', '05:10:00', '05:15:00', '05:20:00', '05:25:00', '05:30:00', '05:35:00', '05:40:00', '05:45:00', '05:50:00', '05:55:00', '06:00:00', '06:05:00', '06:10:00', '06:15:00', '06:20:00', '06:25:00', '06:30:00', '06:35:00', '06:40:00', '06:45:00', '0

In [5]:
file_xd_speed = "/Users/haowu/Desktop/Transportation Research/Traffic-Prediction/data/Cranberry_ritis_1min_class123/manually_select_cranberry_class123_20181101_20190727_dont_average/manually_select_cranberry_class123_20181101_20190727_dont_average.csv"

In [11]:
chunksize = 10 ** 7
# xd_hms = []

# took about 4~5 min to read
with pd.read_csv(file_xd_speed, chunksize=chunksize) as reader:
    chunks = [chunk for chunk in reader] # 62 chunks of (10000000, 8)

        

In [25]:
print(len(chunks))
print(chunks[0].head(5)) 
# time = list(chunks[0]["measurement_tstamp"].unique())
# hms = set([t.split()[1] for t in time])
# print(len(hms))
# print(sorted(list(hms)))



62
        xd_id   measurement_tstamp  speed  average_speed  reference_speed  \
0  1310384989  2018-11-01 00:00:00   64.0           64.0             66.0   
1  1310384989  2018-11-01 00:01:00   64.0           64.0             66.0   
2  1310384989  2018-11-01 00:02:00   59.0           64.0             66.0   
3  1310384989  2018-11-01 00:03:00   58.0           64.0             66.0   
4  1310384989  2018-11-01 00:04:00   65.0           64.0             66.0   

   travel_time_minutes  confidence_score  cvalue  
0                 0.60              30.0    91.0  
1                 0.60              30.0    91.0  
2                 0.65              30.0    58.0  
3                 0.66              30.0    51.0  
4                 0.59              30.0    83.0  


In [ ]:
time = list(temp_xd["measurement_tstamp"].unique())
hms = set([t.split()[1] for t in time])
print(hms)

In [26]:
tmc = pd.read_csv("./data/TMC.csv") # (25372313, 4) # 335个 unique tmc code
xd = pd.read_csv("./data/XD.csv") # (3063534, 4), 42个unique xd_id

In [55]:
print(xd[:5])
print(xd[:-5])

        xd_id   measurement_tstamp  speed  reference_speed
0  1310417514  2018-11-01 00:00:00   35.0             35.0
1  1310417514  2018-11-01 00:05:00   35.0             35.0
2  1310417514  2018-11-01 00:10:00   35.0             35.0
3  1310417514  2018-11-01 00:15:00   35.0             35.0
4  1310417514  2018-11-01 00:20:00   35.0             35.0
              xd_id   measurement_tstamp  speed  reference_speed
0        1310417514  2018-11-01 00:00:00  35.00             35.0
1        1310417514  2018-11-01 00:05:00  35.00             35.0
2        1310417514  2018-11-01 00:10:00  35.00             35.0
3        1310417514  2018-11-01 00:15:00  35.00             35.0
4        1310417514  2018-11-01 00:20:00  35.00             35.0
...             ...                  ...    ...              ...
3063524  1310597496  2019-07-27 17:45:00  40.20             40.0
3063525  1310597496  2019-07-27 17:50:00  47.39             40.0
3063526  1310597496  2019-07-27 17:55:00  47.00             4

In [54]:
print(tmc[:5])
print(tmc[:-5])

    tmc_code   measurement_tstamp  speed  reference_speed
0  104+04440  2018-11-01 00:00:00   60.0             61.0
1  104+04440  2018-11-01 00:05:00   60.0             61.0
2  104+04440  2018-11-01 00:10:00   60.0             61.0
3  104+04440  2018-11-01 00:15:00   60.6             61.0
4  104+04440  2018-11-01 00:20:00   61.0             61.0
           tmc_code   measurement_tstamp  speed  reference_speed
0         104+04440  2018-11-01 00:00:00  60.00             61.0
1         104+04440  2018-11-01 00:05:00  60.00             61.0
2         104+04440  2018-11-01 00:10:00  60.00             61.0
3         104+04440  2018-11-01 00:15:00  60.60             61.0
4         104+04440  2018-11-01 00:20:00  61.00             61.0
...             ...                  ...    ...              ...
25372303  104P11612  2019-07-27 16:10:00  53.44             44.0
25372304  104P11612  2019-07-27 16:15:00  53.33             44.0
25372305  104P11612  2019-07-27 16:20:00  45.61             44.0
25

In [59]:
target_tmcs = np.load('./data/cran_tmc.npy',allow_pickle=True) # 162 segments
X = np.load('./data/X.npy') # (29520, 7, 1008), first dim denotes 12 slots * 15 hrs * 164 days (2019.2.10~2019.7.23)
Y = np.load('./data/Y.npy') # (29520, 7, 162)
col_names = list(np.load("/Users/haowu/Desktop/Transportation Research/FHWA/zsl_cranberry/data/col_names.npy", allow_pickle=True))

In [61]:
print(col_names)

['104+04438', '104+04439', '104+04440', '104+04441', '104+04442', '104+04443', '104+04444', '104+04445', '104+04520', '104+04521', '104+04522', '104+04523', '104+04524', '104+04525', '104+04526', '104+04527', '104+04528', '104+04529', '104+04530', '104+04531', '104+04532', '104+04533', '104+04540', '104+04541', '104+04542', '104+04543', '104+04544', '104+04636', '104+04637', '104+04639', '104+04640', '104+04641', '104+04642', '104+04643', '104+04644', '104+04734', '104+04735', '104+04736', '104+04737', '104+04738', '104+04739', '104+04740', '104+04741', '104+04742', '104+04782', '104+04783', '104+04784', '104+04785', '104+04800', '104+04801', '104+06224', '104+06225', '104+06226', '104+06227', '104+06228', '104+06229', '104+06230', '104+06592', '104+06676', '104+06678', '104+06679', '104+06680', '104+06681', '104+06682', '104+06683', '104+06684', '104+06685', '104+08678', '104+08679', '104+09013', '104+09014', '104+09017', '104+09018', '104+09193', '104+09194', '104+09195', '104+09196'

In [62]:
feat_weather_time = col_names[-21:]

In [69]:
feat_speed = [i for i in col_names[:-21] if "sd" not in i and "inc" not in i] # 369
feat_sd = [i for i in col_names[:-21] if "sd" in i] # 303
feat_inc = [i for i in col_names[:-21] if "inc" in i] # 315

In [72]:
print(feat_speed)
print(feat_sd)
print(feat_inc)

['104+04438', '104+04439', '104+04440', '104+04441', '104+04442', '104+04443', '104+04444', '104+04445', '104+04520', '104+04521', '104+04522', '104+04523', '104+04524', '104+04525', '104+04526', '104+04527', '104+04528', '104+04529', '104+04530', '104+04531', '104+04532', '104+04533', '104+04540', '104+04541', '104+04542', '104+04543', '104+04544', '104+04636', '104+04637', '104+04639', '104+04640', '104+04641', '104+04642', '104+04643', '104+04644', '104+04734', '104+04735', '104+04736', '104+04737', '104+04738', '104+04739', '104+04740', '104+04741', '104+04742', '104+04782', '104+04783', '104+04784', '104+04785', '104+04800', '104+04801', '104+06224', '104+06225', '104+06226', '104+06227', '104+06228', '104+06229', '104+06230', '104+06592', '104+06676', '104+06678', '104+06679', '104+06680', '104+06681', '104+06682', '104+06683', '104+06684', '104+06685', '104+08678', '104+08679', '104+09013', '104+09014', '104+09017', '104+09018', '104+09193', '104+09194', '104+09195', '104+09196'

In [79]:
seg_sd = [s[3:] for s in feat_sd]
seg_inc = [i[4:] for i in feat_inc]
print(set(seg_sd).issubset(set(seg_inc)))
print(set(seg_inc).issubset(set(feat_speed)))
print(len(set(seg_inc).intersection(set(feat_speed))))

True
False
309


In [20]:
waze = pd.read_pickle('./data/waze_vis.pickle').reset_index(drop=True) # (150468, 7)
rcrs = pd.read_pickle('./data/rcrs_vis.pickle').reset_index(drop=True) # (1511, 19)
weather = pd.read_pickle('./data/weather.pickle').reset_index(drop=True) # (268977, 12)

In [58]:
print (weather[:-5])

                  valid_time_gmt  temp  pressure  pressure_tend  dewPt  wspd  \
0      2017-01-01 00:15:00-05:00  37.0     28.54            0.0   34.0  12.0   
1      2017-01-01 00:20:00-05:00  37.0     28.55            0.0   34.0  13.0   
2      2017-01-01 00:25:00-05:00  37.0     28.55            0.0   34.0  13.0   
3      2017-01-01 00:30:00-05:00  37.0     28.55            0.0   34.0  13.0   
4      2017-01-01 00:35:00-05:00  37.0     28.55            0.0   34.0  13.0   
...                          ...   ...       ...            ...    ...   ...   
268967 2019-07-23 23:10:00-04:00  58.0     28.66            0.0   53.0   0.0   
268968 2019-07-23 23:15:00-04:00  58.0     28.66            0.0   53.0   0.0   
268969 2019-07-23 23:20:00-04:00  58.0     28.66            0.0   53.0   0.0   
268970 2019-07-23 23:25:00-04:00  58.0     28.66            0.0   53.0   0.0   
268971 2019-07-23 23:30:00-04:00  58.0     28.66            0.0   53.0   0.0   

        gust   vis    rh  precip_hrly  

In [56]:
print(waze[:-5])

                    DateTimeEventOccurs      ActualDateTimeOpened  \
0      2019-02-09 17:43:31.055000-05:00 2019-02-09 18:13:00-05:00   
1      2019-02-09 17:45:54.114000-05:00 2019-02-09 18:18:00-05:00   
2      2019-02-09 17:49:29.683000-05:00 2019-02-09 18:18:00-05:00   
3      2019-02-09 17:54:43.322000-05:00 2019-02-09 18:28:00-05:00   
4      2019-02-09 16:46:08.560000-05:00 2019-02-09 18:33:00-05:00   
...                                 ...                       ...   
150458        2019-07-24 18:38:59-04:00 2019-07-24 22:38:00-04:00   
150459        2019-07-24 22:11:58-04:00 2019-07-24 22:38:00-04:00   
150460        2019-07-24 22:25:33-04:00 2019-07-24 22:38:00-04:00   
150461        2019-07-24 22:14:11-04:00 2019-07-24 22:43:00-04:00   
150462        2019-07-24 22:24:10-04:00 2019-07-24 22:53:00-04:00   

                                                   street  \
0                                                  SR-228   
1                                             McK

In [57]:
print(rcrs[:-5])

      EventID DateTimeEventOccurs ActualDateTimeOpened  \
0      436906    2019-02-11 09:22     2019-02-11 12:58   
1      436905    2019-02-11 09:22     2019-02-11 13:02   
2      436907    2019-02-11 10:00     2019-02-11 14:35   
3      436917    2019-02-11 10:54     2019-02-11 17:00   
4      436959    2019-02-11 19:58     2019-02-11 21:06   
...       ...                 ...                  ...   
1501   473451    2019-07-24 08:53     2019-07-24 11:14   
1502   473515    2019-07-24 09:47     2019-07-24 11:34   
1503   473360    2019-07-24 07:13     2019-07-24 13:06   
1504   473545    2019-07-24 11:28     2019-07-24 14:13   
1505   473546    2019-07-24 11:31     2019-07-24 14:25   

                         Facility  StateRouteNo        Direction  \
0                  HIGHLAND PK BR          1005       southbound   
1                  HIGHLAND PK BR          1005       northbound   
2                         I - 279           279       southbound   
3     SAXONBURG RD / SAXONBURG 